In [7]:
import pandas as pd
import gensim
import numpy as np                                                               
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import math
import numpy as np
#df=pd.read_pickle('./pandas_data_candidatos_lda.pd')
#df=pd.read_pickle('/home/kid/Documentos/analisis2018/pandas_data_candidatos_lda.pd')
df=pd.read_pickle('/home/kid/Documentos/analisis2018/pandas_data_candidatos_premas.pd')

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(use_idf=True, norm=None, ngram_range=(1, 1)) 
#X = tfidf.fit_transform(te)
matrix = tfidf.fit_transform(df.Text_clean).toarray()
labels_tfidf=list(df.abreviatura)
vectors_tfidf=list(matrix)

In [9]:
matriz_tfidf=np.asarray(vectors_tfidf) #conver in matriz
matriz_tfidf.shape

(900, 55788)

In [10]:
index = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(matriz_tfidf.T))
similarity_matrix_tfidf = []
for sims in index:
    similarity_matrix_tfidf.append(sims)
similarity_matrix_tfidf = np.array(similarity_matrix_tfidf)

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [11]:
def most_simi(doc,num=10):
    pos=labels_tfidf.index(doc)
    simis=list(similarity_matrix_tfidf[pos][:])
    zipsimi=list(zip(labels_tfidf,simis))
    zipsimi.sort(key=lambda x: x[1],reverse=True)
    return zipsimi[1:num+1]

In [12]:
def most_simi_can(doc1,doc2):
    pos1=labels_tfidf.index(doc1)
    pos2=labels_tfidf.index(doc2)
    return similarity_matrix_tfidf[pos1][pos2]

## ----------------------------------------------------

In [13]:
from gensim.models.doc2vec import Doc2Vec
model_d2v = Doc2Vec.load('./modeloDoc2vec_setembro_premas40')
labels_d2v = list(model_d2v.docvecs.doctags.keys())

In [14]:
import pandas as pd
from operator import itemgetter
from collections import Counter

df=pd.read_pickle('/home/kid/Documentos/analisis2018/pandas_data_candidatos_premas.pd')
labels=list(df[df.Weights_kbs>10000]['abreviatura'])
print(df.shape)
print(len(labels))
labels_d2v=labels

(900, 8)
656


In [20]:
from itertools import groupby
def similarity_camada1tf(labels,model,opc):
    #Preparando dados
    nomes=[]
    for t in labels:
        nomes.append(t[0:4])
    ct=Counter(nomes)
    listoTodo = sorted(labels, key=itemgetter(0,1,2))
    indices=['Alva','Bols','Ciro','Gera','Guil','Henr','Joao','Mari']
    ll_candidatos = [list(value) for key, value in groupby(listoTodo, lambda x: x[0:4])]
    general=[]
    for dis_candidato in ll_candidatos:
        tmp=[]
        for x in dis_candidato:
            for t in listoTodo:
                if x[0:4]!=t[0:4] and x[0:4] in indices and t[0:4] in indices:
                    if opc == 1:
                        tmp.append((x,t,most_simi_can(x,t)))
                    else:
                        tmp.append((x,t,model_d2v.docvecs.similarity(x,t)))
                    #print(x,t)
        general.append(tmp)
    return general

In [35]:
cm1= similarity_camada1tf(labels_d2v,model_d2v,2)
camada2=[]
for candidatox in cm1:
    sort_canx=sorted(candidatox, key=lambda tup: tup[1])
    cam2=[list(v) for k, v in groupby(sort_canx, lambda x: x[1][0:4])]
    #stl=stl.sort(key=lambda tup: tup[2])
    camada2.append(cam2)

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [36]:
moda=50
for cm1 in camada2:
    for x in cm1:
        #print('---------------------------------')
        x.sort(key=lambda tup: tup[2], reverse=True)
        mn=sum([v[2] for v in x[:moda]]) / float(len(x[:moda]))
        print(x[0][0][0:4]+' -> '+x[0][1][0:4]+' = '+str(mn))
    print('---------------------------------') 

Alva -> Bols = 0.4934045034646988
Alva -> Ciro = 0.5146043282747269
Alva -> Gera = 0.49251489877700805
Alva -> Guil = 0.5022688138484955
Alva -> Henr = 0.4826799011230469
Alva -> Joao = 0.4984178376197815
Alva -> Mari = 0.48645084738731387
---------------------------------
Bols -> Alva = 0.4934045034646988
Bols -> Ciro = 0.5177497816085815
Bols -> Gera = 0.48959239780902863
Bols -> Guil = 0.48886500775814057
Bols -> Henr = 0.4703986895084381
Bols -> Joao = 0.4895739722251892
Bols -> Mari = 0.4871939063072205
---------------------------------
Ciro -> Alva = 0.5146043282747269
Ciro -> Bols = 0.5177497816085815
Ciro -> Gera = 0.5285866248607636
Ciro -> Guil = 0.5206684803962708
Ciro -> Henr = 0.4904327040910721
Ciro -> Joao = 0.5195780324935914
Ciro -> Mari = 0.5121679502725601
---------------------------------
---------------------------------
Gera -> Alva = 0.49251489877700805
Gera -> Bols = 0.48959239780902863
Gera -> Ciro = 0.5285866248607636
Gera -> Guil = 0.5011965650320053
Gera -> 